In [0]:
import torch
import pdb
import torch.nn as nn
import math
from torch.autograd import Variable
from torch.autograd import Function
import time
from torch.distributions.relaxed_bernoulli import RelaxedBernoulli
import numpy as np


def Binarize(tensor,quant_mode='det'):
    if quant_mode=='det':
        return tensor.sign()
    else:
        return tensor.add_(1).div_(2).add_(torch.rand(tensor.size()).add(-0.5)).clamp_(0,1).round().mul_(2).add_(-1)

def sample2(mu, log_sigma2):
    eps = torch.randn_like(mu)
    s = mu + torch.exp(log_sigma2 / 2) * eps
    return s
  
  

def sampling(mu,sig):
  x = Normal(mu,sig)
#   x = x.sample(torch.tensor([out_features]))
  p = 1 - x.cdf(0)
  p = Binarize(p)
  m = RelaxedBernoulli(torch.tensor([1.]).cuda(),p)
  x = m.sample()
  return x



import torch.nn._functions as tnnf



      
# def pre(mu,sigma):
      
#       m = mu.mean(1).unsqueeze(1)
#       v = sigma.var(1).unsqueeze(1)
#       mu = 0.5*(mu-m)/((v+(1e-6)).sqrt()+0.5)
#       sigma = 0.5**2*sigma/(v+0.5)
      
#       return mu,sigma

    
class PBinarizeLinear(nn.Linear):

    def __init__(self, *kargs, **kwargs):
        super(PBinarizeLinear, self).__init__(*kargs, **kwargs)
        

    def forward(self, input):
        

        if input.size(1) != 784:
            input.data=Binarize(input.data)
            
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()  
            
        self.weight.data=Binarize(self.weight.org)
#         print(self.weight.shape)
        
#         p = np.count_nonzero((self.weight.data.cpu()+1)/2)/np.count_nonzero(self.weight.data.cpu())
#         print(p)
        
        mu = self.weight.data.std()/self.weight.data
        var = (self.weight.data - mu)**2*mu

       
        mu = nn.functional.linear(input,mu)   
        sigma = nn.functional.linear(input**2,var)
        

        m = mu.mean()
        v = sigma.var()
       
        
        
        mu = 0.5*(mu-m)/((v+(0.5)).sqrt()+0.5)
        sigma = 0.5**2*sigma/(v+0.5)
        

       
        out1 = sampling(mu,sigma)
#         out = nn.functional.linear(input, self.weight)

#         if not self.bias is None:
#             self.bias.org=self.bias.data.clone()
#             out += self.bias.view(1, -1).expand_as(out)

        return out1






In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from tqdm import tqdm
from torch.distributions.normal import Normal
from torch.distributions.relaxed_bernoulli import RelaxedBernoulli
from torch.distributions.relaxed_categorical import RelaxedOneHotCategorical

from torch.distributions.categorical import Categorical

torch.manual_seed(1)
# if args.cuda:
#     torch.cuda.manual_seed(args.seed)


# kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True)





# 32C3 - MP2 - 64C3 - Mp2 - 512FC - SM10c
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
#         self.conv1 = PBinarizeConv2d(1, 32, kernel_size=3)
        
#         self.mp1 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.htanh1 = nn.Hardtanh()
        
#         self.conv2 = PBinarizeConv2d(32, 64, kernel_size=3)
#         self.mp2 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.htanh2 = nn.Hardtanh()
        
        self.fc1 = PBinarizeLinear(784, 512)
#         self.htanh3 = nn.Hardtanh()
        
        self.fc2 = nn.Linear(512, 10)


    # 32C3 - MP2 - 64C3 - Mp2 - 512FC - SM10c
  
    def forward(self, x):
      
 
        x = x.view(x.size(0), -1)
#         print(x.size())
       
        x = self.fc1(x)
       
        x = self.fc2(x)

        
        return x
  


model = Net()

print(model)

torch.cuda.device('cuda')
model.cuda()


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


def train(epoch):
    model.train()
    
    losses = []
    trainloader = tqdm(train_loader)
    
    for batch_idx, (data, target) in enumerate(trainloader):
 
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
#         print()
#         print(loss)

#         if epoch%40==0:
#             optimizer.param_groups[0]['lr']=optimizer.param_groups[0]['lr']*0.1

#         optimizer.zero_grad()
#         
        loss.backward()
    
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.data.copy_(p.org)
        optimizer.step()
        
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.org.copy_(p.data.clamp_(-0.9,0.9))
    
        losses.append(loss.item())
        trainloader.set_postfix(loss=np.mean(losses), epoch=epoch)



def test():
    model.eval()
    test_loss = 0
    correct = 0
    testloader = tqdm(test_loader)
    for data, target in testloader:
        data, target = data.cuda(), target.cuda()
        with torch.no_grad():
          data = Variable(data)
        target = Variable(target)
        output = model(data)
        test_loss += criterion(output, target).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        

        testloader.set_postfix(loss=test_loss / len(test_loader.dataset),acc=str((100. *correct / len(test_loader.dataset)).numpy())+'%')
    
    test_loss /= len(test_loader.dataset)
    
    




Net(
  (fc1): PBinarizeLinear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


In [3]:
%%%time
for epoch in range(5):
    train(epoch)
    test()

 26%|██▌       | 120/469 [00:04<00:13, 26.81it/s, epoch=3, loss=0.739]

KeyboardInterrupt: ignored

In [0]:
a = torch.randn(5,4)
a

In [0]:
a=Binarize(a)
a

In [0]:
(a+1)/2

In [0]:
torch.mean(a, (0), True)

In [0]:
p= np.count_nonzero((a+1)/2,axis=0)/np.count_nonzero(a,axis=0)

In [0]:
1-(p)**2

In [0]:
np.count_nonzero((a+1)/2)

In [0]:
mu = torch.randn(5)
sig = torch.randn(5)


In [0]:
x=Normal(mu,sig)
1 - x.cdf(0)

In [0]:
m = -4.6
v = 25936
x = Normal(m,v)
p = 1 - x.cdf(0)
# s = sample_gumbel_softmax(p,1.0)



In [0]:
eps = torch.randn_like(mu)
    s = mu + torch.exp(log_sigma2 / 2) * eps

In [0]:
p.sample()

In [0]:
aa = Normal(m,v)
# aa.sample(torch.tensor([20]))

In [0]:
1 - x.cdf(0)

In [0]:
m = RelaxedOneHotCategorical(torch.tensor([1.]),a)
m.sample()

In [0]:
""# # Execute this code block to install dependencies when running on colab
# try:
#     import torch
# except:
#     from os.path import exists
#     from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
#     platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
#     cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
#     accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

#     !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

# try: 
#     import torchbearer
# except:
#     !pip install torchbearer
    
# from torchbearer import Trial
# torchbearer_trial = Trial(model, optimizer, criterion, metrics=['loss', 'accuracy']).to('cuda:0')
# torchbearer_trial.with_generators(train_loader, test_generator=test_loader)
# torchbearer_trial.run(epochs=5)